In [65]:
import numpy as np
import csv
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from libsvm.python.svmutil import *

In [2]:
def loadData():
    X_train = np.genfromtxt('X_train.csv', delimiter=',')
    Y_train = np.genfromtxt('Y_train.csv', dtype=np.int)
    X_test = np.genfromtxt('X_test.csv', delimiter=',')
    Y_test = np.genfromtxt('Y_test.csv', dtype=np.int)
    return X_train, Y_train, X_test, Y_test

# Load Data

In [3]:
X_train, Y_train, X_test, Y_test = loadData()

# Part 1

In [65]:
kernelParams = {'linear': '-t 0', 'polynomial': '-t 1', 'RBF': '-t 2'}

In [66]:
acc_test = {}
for kernelType, param in kernelParams.items():
    model = svm_train(Y_train, X_train, param)
    p_label, p_acc, p_vals = svm_predict(Y_test, X_test, model, '-q')
    acc_test[kernelType] = p_acc[0]

In [67]:
print('Testing Accuracy:')
for kernelType, acc in acc_test.items():
    print(f'{kernelType:>10s}: {acc:.2f} %')

Testing Accuracy:
    linear: 95.08 %
polynomial: 34.68 %
       RBF: 95.32 %


# Part 2

In [75]:
def gridSearch(X_train, Y_train, X_test, Y_test, kernelType: str, kernelFunction: callable=None, **searchSpace):
    # search sapce: search space of each parameter
    # return: ndarray with cross-validation acc of every combinations of parameters in the search space
    
    if kernelType == 'linear': # param: C
        acc_validate = np.zeros(len(searchSpace['C']))
        
        for i_c, c in enumerate(searchSpace['C']):
            acc_validate[i_c] = svm_train(Y_train, X_train, f'-s 0 -t 0 -v 3 -c {c}')
    
    
    elif kernelType == 'polynomial': # param: C, gamma, coef0, degree
        acc_validate = np.zeros((len(searchSpace['C']), len(searchSpace['gamma']), len(searchSpace['coef0']), len(searchSpace['degree'])))
        
        for i_c, c in enumerate(searchSpace['C']):
            for i_gamma, gamma in enumerate(searchSpace['gamma']):
                for i_coef0, coef0 in enumerate(searchSpace['coef0']):
                    for i_degree, degree in enumerate(searchSpace['degree']):
                        acc_validate[i_c, i_gamma, i_coef0, i_degree] = svm_train(Y_train, X_train, f'-s 0 -t 1 -v 3 -c {c} -g {gamma} -r {coef0} -d {degree}')
    
    
    elif kernelType == 'RBF': # param: C, gamma
        acc_validate = np.zeros((len(searchSpace['C']), len(searchSpace['gamma'])))
        
        for i_c, c in enumerate(searchSpace['C']):
            for i_gamma, gamma in enumerate(searchSpace['gamma']):
                acc_validate[i_c, i_gamma] = svm_train(Y_train, X_train, f'-s 0 -t 2 -v 3 -c {c} -g {gamma}')
    
    
    elif kernelType == 'custom': # param: C, gamma
        acc_validate = np.zeros((len(searchSpace['C']), len(searchSpace['gamma'])))
        
        for i_c, c in enumerate(searchSpace['C']):
            for i_gamma, gamma in enumerate(searchSpace['gamma']):
                # compute kernels
                kernel_train = kernelFunction(X_train, X_train, gamma=gamma)
                
                # train with cross-validation
                prob = svm_problem(Y_train, kernel_train, isKernel=True)
                acc_validate[i_c, i_gamma] = svm_train(prob, f'-s 0 -t 4 -v 3 -c {c}')
    
    
    return acc_validate

## Grid Search on Linear Kernel

In [47]:
searchSpace_C = [2 ** i for i in range(-3, 4)]

In [48]:
%%capture
# suppress output of svm_train

acc_validate = gridSearch(X_train, Y_train, X_test, Y_test,
                          kernelType = 'linear',
                          C = searchSpace_C)

In [49]:
acc = np.max(acc_validate)
i = np.argmax(acc_validate)
print(f'Highest validation acc: {acc} %')
print('Parameters:')
print(f'C\t: {searchSpace_C[i]}')

Highest validation acc: 96.64 %
Parameters:
C	: 0.125


In [50]:
np.save('linear_validate', acc_validate) # save validation result

### Train / Test

In [55]:
acc_validate = np.load('linear_validate.npy') # load validation result
i = np.argmax(acc_validate)

In [56]:
c = searchSpace_C[i]

model = svm_train(Y_train, X_train, f'-s 0 -t 0 -c {c}')
p_label, p_acc, p_vals = svm_predict(Y_test, X_test, model, '-q')
print(f'Testing Accuracy: {p_acc[0]:.2f} %')

Testing Accuracy: 95.92 %


## Grid Search on Polynomial Kernel

In [36]:
searchSpace_C = [2 ** i for i in range(-3, 4)]
searchSpace_gamma = [2 ** i / 784 for i in range(-3, 4)]
searchSpace_coef0 = [-1, 0, 1]
searchSpace_degree = [2, 3, 5, 8]

In [37]:
%%capture
acc_validate = gridSearch(X_train, Y_train, X_test, Y_test,
                          kernelType='polynomial',
                          C=searchSpace_C,
                          gamma=searchSpace_gamma,
                          coef0=searchSpace_coef0,
                          degree=searchSpace_degree)

In [39]:
acc = np.max(acc_validate)
i_c, i_gamma, i_coef0, i_degree = np.unravel_index(np.argmax(acc_validate), (7, 7, 3, 4))
print(f'Highest validation acc: {acc} %')
print('Parameters:')
print(f'C\t: {searchSpace_C[i_c]}')
print(f'gamma\t: {searchSpace_gamma[i_gamma]}')
print(f'coef0\t: {searchSpace_coef0[i_coef0]}')
print(f'degree\t: {searchSpace_degree[i_degree]}')

Highest validation acc: 98.11999999999999 %
Parameters:
C	: 8
gamma	: 0.01020408163265306
coef0	: 0
degree	: 2


In [44]:
np.save('polynomial_validate.npy', acc_validate)

### Train / Test

In [58]:
acc_validate = np.load('polynomial_validate.npy') # load validation result
i_c, i_gamma, i_coef0, i_degree = np.unravel_index(np.argmax(acc_validate), (7, 7, 3, 4))

In [59]:
c = searchSpace_C[i_c]
gamma = searchSpace_gamma[i_gamma]
coef0 = searchSpace_coef0[i_coef0]
degree = searchSpace_degree[i_degree]

model = svm_train(Y_train, X_train, f'-s 0 -t 1 -c {c} -g {gamma} -r {coef0} -d {degree}')
p_label, p_acc, p_vals = svm_predict(Y_test, X_test, model, '-q')
print(f'Testing Accuracy: {p_acc[0]:.2f} %')

Testing Accuracy: 97.84 %


## Grid Search on RBF Kernel

In [51]:
searchSpace_C = [2 ** i for i in range(-3, 4)]
searchSpace_gamma = [2 ** i / 784 for i in range(-3, 4)]

In [52]:
%%capture
acc_validate = gridSearch(X_train, Y_train, X_test, Y_test,
                          kernelType='RBF',
                          C=searchSpace_C,
                          gamma=searchSpace_gamma)

In [53]:
acc = np.max(acc_validate)
i_c, i_gamma = np.unravel_index(np.argmax(acc_validate), (7, 7))
print(f'Highest validation acc: {acc} %')
print('Parameters:')
print(f'C\t: {searchSpace_C[i_c]}')
print(f'gamma\t: {searchSpace_gamma[i_gamma]}')

Highest validation acc: 98.18 %
Parameters:
C	: 8
gamma	: 0.01020408163265306


In [54]:
np.save('RBF_validate', acc_validate)

### Train / Test

In [61]:
acc_validate = np.load('RBF_validate.npy') # load validation result
i_c, i_gamma = np.unravel_index(np.argmax(acc_validate), (7, 7))

In [62]:
c = searchSpace_C[i_c]
gamma = searchSpace_gamma[i_gamma]

model = svm_train(Y_train, X_train, f'-s 0 -t 2 -c {c} -g {gamma}')
p_label, p_acc, p_vals = svm_predict(Y_test, X_test, model, '-q')
print(f'Testing Accuracy: {p_acc[0]:.2f} %')

Testing Accuracy: 98.20 %


# Part 3

In [72]:
def computeCustomKernel(U, V, gamma):
    # u, v: Nxm
    kernel_linear = U @ V.T
    kernel_RBF = np.exp(-gamma * cdist(U, V, 'sqeuclidean'))
    kernel = kernel_linear + kernel_RBF
    kernel = np.hstack((np.arange(1, U.shape[0] + 1).reshape(-1, 1), kernel))
    return kernel

In [73]:
# train
kernel_train = computeCustomKernel(X_train, X_train, gamma=1/784) # default gamma
prob = svm_problem(Y_train, kernel_train, isKernel=True)
model = svm_train(prob, '-s 0 -t 4')

# test
kernel_test = computeCustomKernel(X_test, X_train, gamma=1/784)
p_label, p_acc, p_vals = svm_predict(Y_test, kernel_test, model, '-q')
print(f'Testing Accuracy: {p_acc[0]:.2f} %')

Testing Accuracy: 95.08 %


## Grid Search on Custom (Linear + RBF) Kernel

In [76]:
searchSpace_C = [2 ** i for i in range(-3, 4)]
searchSpace_gamma = [2 ** i / 784 for i in range(-3, 4)]

In [78]:
%%capture
acc_validate = gridSearch(X_train, Y_train, X_test, Y_test,
                          kernelType='custom',
                          kernelFunction=computeCustomKernel,
                          C=searchSpace_C,
                          gamma=searchSpace_gamma)

In [79]:
acc = np.max(acc_validate)
i_c, i_gamma = np.unravel_index(np.argmax(acc_validate), (7, 7))
print(f'Highest validation acc: {acc} %')
print('Parameters:')
print(f'C\t: {searchSpace_C[i_c]}')
print(f'gamma\t: {searchSpace_gamma[i_gamma]}')

Highest validation acc: 96.88 %
Parameters:
C	: 0.125
gamma	: 0.01020408163265306


In [80]:
np.save('linear_RBF_validate', acc_validate)

### Train / Test

In [81]:
acc_validate = np.load('linear_RBF_validate.npy') # load validation result
i_c, i_gamma = np.unravel_index(np.argmax(acc_validate), (7, 7))

In [82]:
c = searchSpace_C[i_c]
gamma = searchSpace_gamma[i_gamma]

# train
precomputedKernel_train = computeCustomKernel(X_train, X_train, gamma=gamma)
prob = svm_problem(Y_train, precomputedKernel_train, isKernel=True)
model = svm_train(prob, f'-s 0 -t 4 -c {c}')

# test
precomputedKernel_test = computeCustomKernel(X_test, X_train, gamma=gamma)
p_label, p_acc, p_vals = svm_predict(Y_test, precomputedKernel_test, model, '-q')
print(f'Testing Accuracy: {p_acc[0]:.2f} %')

Testing Accuracy: 95.96 %


In [ ]:
plt.figure(figsize=(10, 10))
plt.matshow(acc_validate, cmap='Blues', fignum=1)
plt.gca().set_xticklabels([''] + list(np.round(searchSpace_gamma, 6)))
plt.gca().set_yticklabels([''] + searchSpace_C)
plt.gca().xaxis.set_label_position('top')
plt.gca().yaxis.set_label_position('left')
plt.xlabel('gamma', fontsize=20)
plt.ylabel('C', fontsize=20)
for i_c, c in enumerate(searchSpace_C):
    for i_gamma, gamma in enumerate(searchSpace_gamma):
        plt.text(i_gamma, i_c, f'{acc_validate[i_c, i_gamma]:.2f}', va='center', ha='center', fontsize=14)

plt.show()